### Gamma Ray Prediction

In [3]:
# Step 1: Imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt
import random

In [4]:
# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Step 2: Load Data
df = pd.read_csv('cleaned_well_data.csv')
depth = df['DEPT'].values
input_features = ['SP:1', 'PHIE:1', 'VSH', 'SP:2', 'RES']
target_logs = ['GR']

In [5]:
# Step 3: Scaling
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X = scaler_x.fit_transform(df[input_features])
y = scaler_y.fit_transform(df[target_logs])

# Step 4: Sequence Preparation
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])  # predict next GR after sequence
    return np.array(X_seq), np.array(y_seq)

seq_length = 30  # Adjustable
X_seq, y_seq = create_sequences(X, y, seq_length)

In [6]:
# Step 5: Train-Test Split
# Train-test split (80-20 split)
split_idx = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]
depth_train, depth_test = depth[seq_length:split_idx+seq_length], depth[split_idx+seq_length:]

# Step 6: Physics-Informed Loss
import tensorflow.keras.backend as K

def physics_informed_loss(y_true, y_pred):
    mse = K.mean(K.square(y_true - y_pred))
    smooth_penalty = K.mean(K.square(y_pred[1:] - y_pred[:-1]))  # smoothness in GR
    range_penalty = K.mean(K.maximum(y_pred - 150, 0.0)) + K.mean(K.maximum(0.0 - y_pred, 0.0))
    return mse + 0.2 * smooth_penalty + 0.2 * range_penalty


In [7]:
# Step 7: LSTM Model
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = LSTM(64, return_sequences=True)(input_layer)
x = Dropout(0.2)(x)
x = LSTM(32)(x)
x = Dropout(0.2)(x)
output_layer = Dense(1)(x)
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss=physics_informed_loss, metrics=['mae'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 5)]           0         
                                                                 
 lstm (LSTM)                 (None, 30, 64)            17920     
                                                                 
 dropout (Dropout)           (None, 30, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 30,369
Trainable params: 30,369
Non-trainable p

In [ ]:
# Step 8: Train Model
history = model.fit(
    X_train, y_train,
    epochs=250,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/250
124/124 [==============================] - 21s 62ms/step - loss: 0.0239 - mae: 0.1080 - val_loss: 0.0090 - val_mae: 0.0750
Epoch 2/250
124/124 [==============================] - 5s 41ms/step - loss: 0.0156 - mae: 0.0865 - val_loss: 0.0076 - val_mae: 0.0692
Epoch 3/250
124/124 [==============================] - 6s 46ms/step - loss: 0.0144 - mae: 0.0817 - val_loss: 0.0072 - val_mae: 0.0648
Epoch 4/250
124/124 [==============================] - 5s 42ms/step - loss: 0.0136 - mae: 0.0794 - val_loss: 0.0066 - val_mae: 0.0665
Epoch 5/250
124/124 [==============================] - 5s 41ms/step - loss: 0.0133 - mae: 0.0783 - val_loss: 0.0079 - val_mae: 0.0686
Epoch 6/250
124/124 [==============================] - 5s 44ms/step - loss: 0.0126 - mae: 0.0757 - val_loss: 0.0058 - val_mae: 0.0616
Epoch 7/250
 17/124 [===>..........................] - ETA: 4s - loss: 0.0116 - mae: 0.0732

In [ ]:
# Step 9: Predictions and Metrics
y_pred = model.predict(X_test)
y_test_rescaled = scaler_y.inverse_transform(y_test)
y_pred_rescaled = scaler_y.inverse_transform(y_pred)

mape = mean_absolute_percentage_error(y_test_rescaled, y_pred_rescaled) * 100
accuracy = 100 - mape
r2 = r2_score(y_test_rescaled, y_pred_rescaled)
mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)

print(f"📊 Accuracy: {accuracy:.2f}%")
print(f"📉 MAPE: {mape:.2f}%")
print(f"📈 R²: {r2:.4f}")
print(f"📉 MSE: {mse:.4f}")

In [ ]:
plt.figure(figsize=(8, 10))
plt.plot(y_test_rescaled, depth_seq, label='Actual GR', color='blue')
plt.plot(y_pred_rescaled, depth_seq, label='Predicted GR', color='orange')
plt.gca().invert_yaxis()  # Depth increases downwards
plt.title('Gamma Ray Prediction vs Depth')
plt.xlabel('Gamma Ray (API)')
plt.ylabel('Depth (m)')
plt.legend()
plt.grid(True)
plt.show()